In [ ]:
import os
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import google.generativeai as genai
from qdrant_client import QdrantClient

# Configuração Visual
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

# Carregar variáveis de ambiente
# Tenta carregar do .env.local ou .env na raiz do projeto
env_path = os.path.join(os.path.dirname(os.getcwd()), '.env.local')
if not os.path.exists(env_path):
    env_path = os.path.join(os.getcwd(), '..', '.env.local')

if not os.path.exists(env_path):
    env_path = os.path.join(os.path.dirname(os.getcwd()), '.env')
    if not os.path.exists(env_path):
        env_path = os.path.join(os.getcwd(), '..', '.env')

load_dotenv(env_path)

# Configurações
# Backend roda na porta 3001
API_URL = "http://localhost:3001/api"
GEMINI_KEY = os.getenv("VITE_GEMINI_API_KEY")
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_KEY = os.getenv("QDRANT_API_KEY")
REDIS_URL = os.getenv("UPSTASH_REDIS_REST_URL")
REDIS_TOKEN = os.getenv("UPSTASH_REDIS_REST_TOKEN")

print(f"✅ Ambiente configurado.")
print(f"📍 API URL: {API_URL}")
print(f"🔑 Gemini Key: {'Set' if GEMINI_KEY else 'Not Set'}")
print(f"🗄️ Qdrant URL: {QDRANT_URL}")
print(f"⚡ Redis URL: {REDIS_URL}")

## 1. Healthcheck da API Local
Verifica se a API local (Node.js) está rodando e respondendo.
Certifique-se de rodar `cd backend && npm run dev` no terminal.

In [ ]:
try:
    # O endpoint /health fica na raiz do backend, não em /api/health
    health_url = API_URL.replace("/api", "/health")
    response = requests.get(health_url, timeout=5)
    if response.status_code == 200:
        data = response.json()
        print("✅ API Online!")
        print(json.dumps(data, indent=2))
    else:
        print(f"❌ Erro na API: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Não foi possível conectar. Verifique se o backend está rodando na porta 3001.")
except Exception as e:
    print(f"❌ Erro: {e}")

## 2. Análise de Agentes
Consulta o status atual dos agentes via API.

In [ ]:
try:
    response = requests.get(f"{API_URL}/agents", timeout=5)

    if response.status_code == 200:
        agents = response.json()
        if agents:
            df_agents = pd.DataFrame(agents)
            # Selecionar colunas relevantes se existirem
            cols = [c for c in ['id', 'name', 'status', 'lastActivity', 'model'] if c in df_agents.columns]
            display(df_agents[cols])

            if 'status' in df_agents.columns:
                status_counts = df_agents['status'].value_counts()
                status_counts.plot(kind='bar', color='teal', title='Status dos Agentes')
                plt.show()
        else:
            print("⚠️ Nenhum agente retornado pela API.")
    else:
        print(f"⚠️ Endpoint /agents retornou: {response.status_code}")
        # Fallback para dados mockados se a API não estiver pronta
        print("ℹ️ Usando dados de exemplo para visualização:")
        mock_agents = [
            {"id": "agent-1", "name": "Triagem", "status": "idle", "lastActivity": "2024-01-01T10:00:00Z"},
            {"id": "agent-2", "name": "Petições", "status": "working", "lastActivity": "2024-01-01T10:05:00Z"},
            {"id": "agent-3", "name": "Jurisprudência", "status": "error", "lastActivity": "2024-01-01T09:00:00Z"}
        ]
        df_mock = pd.DataFrame(mock_agents)
        display(df_mock)
except Exception as e:
    print(f"Erro ao consultar agentes: {e}")

## 3. Teste do Google Gemini
Testa a conexão direta com o modelo Gemini usando a chave de API.

In [ ]:
if GEMINI_KEY:
    try:
        genai.configure(api_key=GEMINI_KEY)
        model = genai.GenerativeModel('gemini-pro') # Ou use o modelo definido no .env

        prompt = "Explique brevemente o conceito de 'Habeas Corpus' para um leigo."
        print(f"🤖 Enviando prompt: '{prompt}'...")

        response = model.generate_content(prompt)
        print("\n📝 Resposta do Gemini:")
        print(response.text)
    except Exception as e:
        print(f"❌ Erro ao conectar com Gemini: {e}")
else:
    print("⚠️ VITE_GEMINI_API_KEY não encontrada no .env.local")

## 4. Inspeção do Qdrant (Vector DB)
Conecta ao Qdrant e lista as coleções disponíveis.

In [ ]:
if QDRANT_URL and QDRANT_KEY:
    try:
        client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY)
        collections = client.get_collections()

        print(f"📚 Coleções encontradas: {len(collections.collections)}")
        for col in collections.collections:
            print(f" - {col.name}")

            # Tenta pegar info da coleção
            try:
                info = client.get_collection(col.name)
                print(f"   Items: {info.points_count}, Vetores: {info.config.params.vectors}")
            except:
                pass
    except Exception as e:
        print(f"❌ Erro ao conectar com Qdrant: {e}")
else:
    print("⚠️ QDRANT_URL ou QDRANT_API_KEY não configurados.")

## 5. Inspeção do Redis (Upstash)
Verifica chaves no Redis via REST API.

In [ ]:
if REDIS_URL and REDIS_TOKEN:
    try:
        # Usando REST API do Upstash
        headers = {"Authorization": f"Bearer {REDIS_TOKEN}"}
        # Comando DBSIZE
        resp = requests.post(f"{REDIS_URL}/dbsize", headers=headers)

        if resp.status_code == 200:
            result = resp.json()
            print(f"📦 Total de chaves no Redis: {result.get('result')}")

            # Listar algumas chaves (SCAN)
            resp_scan = requests.post(f"{REDIS_URL}/scan", headers=headers, json=["0"])
            if resp_scan.status_code == 200:
                keys = resp_scan.json().get('result', [])[1]
                print(f"🔑 Algumas chaves: {keys}")
        else:
            print(f"❌ Erro Redis: {resp.status_code} - {resp.text}")

    except Exception as e:
        print(f"❌ Erro ao conectar com Redis: {e}")
else:
    print("⚠️ UPSTASH_REDIS_REST_URL ou TOKEN não configurados.")

## 1. Healthcheck do Sistema
Verifica se a API local e os serviços conectados (Redis, Qdrant) estão respondendo.

In [ ]:
try:
    response = requests.get(f"{API_URL}/health")
    if response.status_code == 200:
        data = response.json()
        print("✅ API Online!")
        print(json.dumps(data, indent=2))
    else:
        print(f"❌ Erro na API: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Não foi possível conectar. Verifique se 'npm run dev:with-api' está rodando.")

## 2. Análise de Agentes
Consulta o status atual dos agentes e suas últimas execuções.

In [ ]:
try:
    # Endpoint simulado para listar agentes (ajuste conforme sua API real)
    response = requests.get(f"{API_URL}/agents")

    if response.status_code == 200:
        agents = response.json()
        df_agents = pd.DataFrame(agents)
        if not df_agents.empty:
            display(df_agents[['id', 'name', 'status', 'lastActivity']])

            # Gráfico de Status
            status_counts = df_agents['status'].value_counts()
            status_counts.plot(kind='bar', color='teal', title='Status dos Agentes')
            plt.show()
        else:
            print("⚠️ Nenhum agente encontrado.")
    else:
        print(f"⚠️ Endpoint /agents retornou: {response.status_code}")
except Exception as e:
    print(f"Erro: {e}")